In [33]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
# os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"

import keras
import numpy as np
from keras.models import Model, load_model
from keras.layers import Dense, Input, MaxPooling2D, Conv2D, AveragePooling2D, Activation
from keras.layers import BatchNormalization, Concatenate, Add, Flatten, Dropout
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.datasets import cifar10
from keras.utils import np_utils

In [13]:
(xx_train, yy_train), (x_test, y_test) = cifar10.load_data()
x_train = xx_train[:40000]
y_train = yy_train[:40000]
x_valid = xx_train[40000:50000]
y_valid = yy_train[40000:50000]

x_train = x_train/255.0
x_valid = x_valid/255.0
x_test = x_test/255.0

y_train = np_utils.to_categorical(y_train)
y_valid = np_utils.to_categorical(y_valid)
y_test = np_utils.to_categorical(y_test)s

In [4]:
x_train.shape[1:]

(32, 32, 3)

In [59]:
def bn_relu_conv(filters, size, strides, inp):
    tmp = BatchNormalization()(inp)
    tmp = Activation('relu')(tmp)
    return Conv2D(filters, size, strides=strides, padding='same', kernel_initializer='he_normal')(tmp)

def resnet_block(conv_val, x, repeat_num):
    x = MaxPooling2D(pool_size=(3,3), strides=2)(x)
    for i in xrange(repeat_num//2):
        xtmp = bn_relu_conv(conv_val, (3,3), 1, x)
        xtmp = bn_relu_conv(conv_val, (3,3), 1, xtmp)
        x = Add()([x,xtmp])
    x = Conv2D(conv_val*2, (1,1), strides=1)(x)
    return x

In [70]:
l = Input(x_train.shape[1:])

x = Conv2D(16, (3,3), strides=2, padding='same', kernel_initializer='he_normal')(l)
x = BatchNormalization()(x)
x = Activation('relu')(x)

repeat_val = 4
x = resnet_block(16, x, repeat_val)
x = resnet_block(32, x, repeat_val)
x = resnet_block(64, x, repeat_val)

x = BatchNormalization()(x)
x = Activation('relu')(x)
x = GlobalAveragePooling2D()(x)
# x = AveragePooling2D(pool_size=(2,2))(x)
# x = Flatten()(x)

x = Dense(10, activation='softmax', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(x)

early=EarlyStopping(monitor='val_acc', patience=15, verbose=1, mode='auto')
filepath="weights.best.hdf5"
checkpoint=ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [71]:
model = Model(l,x)
# opt = SGD(lr=0.01, momentum=0.9, decay=0.0001, nesterov=True)
model.compile(
    optimizer='adam',
#     optimizer=opt, 
#               loss='sparse_categorical_crossentropy', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_352 (Conv2D)             (None, 16, 16, 16)   448         input_31[0][0]                   
__________________________________________________________________________________________________
batch_normalization_614 (BatchN (None, 16, 16, 16)   64          conv2d_352[0][0]                 
__________________________________________________________________________________________________
activation_614 (Activation)     (None, 16, 16, 16)   0           batch_normalization_614[0][0]    
__________________________________________________________________________________________________
max_poolin

In [ ]:
#if training was cut short
model = load_model('weights.best.hdf5')

In [72]:
history = model.fit(x_train,y_train,
          batch_size=2000, epochs=150,
          shuffle=True,
          validation_data=(x_valid, y_valid), 
          callbacks=[checkpoint, early])


Train on 40000 samples, validate on 10000 samples
Epoch 1/150
40000/40000 [==============================] - 262s 7ms/step - loss: 2.1304 - acc: 0.2188 - val_loss: 9.7582 - val_acc: 0.1003
Epoch 2/150
40000/40000 [==============================] - 123s 3ms/step - loss: 1.7036 - acc: 0.3710 - val_loss: 2.8636 - val_acc: 0.1039
Epoch 3/150
22000/40000 [===============>..............] - ETA: 47s - loss: 1.5412 - acc: 0.4316

KeyboardInterrupt: 

In [26]:
res = model.evaluate(x_test, y_test, batch_size=512)

10000/10000 [==============================] - 4s 371us/step


In [27]:
print('Loss: %.2f Accuracy: %.2f%%' % (res[0], res[1]*100.0))

Loss: 1.53 Accuracy: 65.17%
